In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import xlsxwriter
# to import xlsxwriter goto command prompt and run "pip install xlsxwriter"
import os

In [2]:
# function to convert dataframe to csv
def conv_csv(loc,df, name):
    with open(loc +name+'.csv', 'wb') as f:
        df.to_csv(f,index = 'false',encoding='cp1252')

In [3]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)        

In [76]:
# Defining paths and inputs
loc_equip = r'C:\Users\611346962\Downloads\Project\Inputs\T_Node_extract_equipment.csv'
loc_trans = r"C:\Users\611346962\Downloads\Project\Inputs\T_Node_extract_Transmedia.csv"

ins_cable_loc = r"C:\Users\611346962\Downloads\Project\Inputs\T node_Connectivity_INS Base Data_Round 3(1).xlsx"
ins_location_loc = r"C:\Users\611346962\Downloads\Project\Inputs\T node_Location_INS Base Data_Round 3(1).xlsx"

loc = r'C:\Users\611346962\Downloads\Project\Inputs\dataframes\\'
o_loc = r'C:\Users\611346962\Downloads\Project\Inputs\output\\'
c_loc = r'C:\Users\611346962\Downloads\Project\Inputs\chunks\\'
create_directory(loc)
create_directory(o_loc)
create_directory(c_loc)
o_file = r"C:\Users\611346962\Downloads\Project\Inputs\\"

In [5]:
%%time
# Reading PIPER data
trans_df = pd.read_csv(loc_trans, error_bad_lines=False,encoding='cp1252')
equip_df = pd.read_csv(loc_equip, error_bad_lines=False,encoding='cp1252')

# equip_df = equip_df[['BT_PLANT_1141_CODE','EMINX','EMINY','EQUIPMENT_NAME','EXCHANGE_1141_CODE']]
equip_df = equip_df[['BT_PLANT_1141_CODE','EMINX','EMINY','EQUIPMENT_NAME']]
trans_df = trans_df[['FROM_EQUIPMENT_NAME','TO_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION']]

Skipping line 177178: expected 11 fields, saw 12

Skipping line 1180008: expected 11 fields, saw 12
Skipping line 1215412: expected 11 fields, saw 12

Skipping line 1588330: expected 11 fields, saw 12

<string>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 10.1 s


In [6]:
equip_df_dup =  equip_df[equip_df['BT_PLANT_1141_CODE'].duplicated()].sort(['BT_PLANT_1141_CODE'])

ignore_list = equip_df_dup.index.tolist()
# equip_df[~equip_df.isin()].shape

equip_unique_df = equip_df[~equip_df.index.isin(ignore_list)]

C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [7]:
# conv_csv(loc,equip_df_dup,'equip_df_dup')
# conv_csv(loc,equip_unique_df,'equip_unique_df')
# conv_csv(loc,equip_df,'equip_df')
# conv_csv(loc,trans_df,'trans_df')

In [8]:
%%time
# Creating PIPER dataframes

# creating transmedia dataframe
trans_df = trans_df.dropna(axis=0, how='all')

# conv_csv(loc,trans_df,'trans_dedup_df')

# creating terminating df from piper
Terminating_df = trans_df[['TO_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION']]

Terminating_df = Terminating_df.rename(columns = {'TO_EQUIPMENT_NAME':'EQUIPMENT_NAME'})

# creating originating df from piper
Originating_df = trans_df[['FROM_EQUIPMENT_NAME','BT_CONDUCTOR_NUMBER','BT_CABLE_NAME','BT_CABLE_DESIGNATION','BT_CABLE_NAME_DESIGNATION']]

Originating_df = Originating_df.rename(columns = {'FROM_EQUIPMENT_NAME':'EQUIPMENT_NAME'})


Wall time: 1.13 s


In [9]:
%%time
# Reading and Creating INS dataframes

# creating INS station dataframe
ins_STTN_df = pd.read_excel(ins_location_loc, sheetname = 'Location',encoding='cp1252')
ins_STTN_df = ins_STTN_df[['STTN_1141_CODE','STTN_EASTING','STTN_NORTHING']]
ins_STTN_df['STTN_EASTING'] = ins_STTN_df['STTN_EASTING'].astype(np.int64)
ins_STTN_df['STTN_NORTHING'] = ins_STTN_df['STTN_NORTHING'].astype(np.int64)

# creating INS cable dataframe originating and terminating seprately
ins_cable_df = pd.read_excel(ins_cable_loc,sheetname = 'Connectivity',encoding='cp1252')
ins_cable_df = ins_cable_df[['Cable Name','SubarCJ','Total Fibres','Originating','Terminating']]

ins_ter_df = ins_cable_df[['Cable Name','SubarCJ','Total Fibres','Terminating']]
ins_org_df = ins_cable_df[['Cable Name','SubarCJ','Total Fibres','Originating']]
ins_ter_df['Total Fibres'] = ins_ter_df['Total Fibres'].convert_objects(convert_numeric=True)
ins_org_df['Total Fibres'] = ins_org_df['Total Fibres'].convert_objects(convert_numeric=True)

ins_ter_df = pd.merge(ins_STTN_df,ins_ter_df,left_on = 'STTN_1141_CODE',right_on = 'Terminating', how = 'left')
ins_org_df = pd.merge(ins_STTN_df,ins_org_df,left_on = 'STTN_1141_CODE',right_on = 'Originating', how = 'left')

C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Wall time: 2min 58s


In [10]:
bt_plant_ = pd.Series()
bt_plant_ = bt_plant_.append(equip_df['BT_PLANT_1141_CODE'])
bt_plant_ = bt_plant_.append(ins_STTN_df['STTN_1141_CODE'])

bt_plant_df = pd.DataFrame()
bt_plant_df['plant_code'] = bt_plant_

bt_plant = bt_plant_df.dropna().drop_duplicates()['plant_code'].tolist()

In [11]:
# Spliting Geometry details in PIPER data
equip_final_df = equip_unique_df

equip_final_df.rename(columns={'EMINX': 'GEOMETRY_Easting', 'EMINY': 'GEOMETRY_Northing'}, inplace=True)

equip_final_df['GEOMETRY_Easting'] = equip_final_df['GEOMETRY_Easting'].convert_objects(convert_numeric=True).apply(lambda x: x/10)

equip_final_df['GEOMETRY_Northing'] = equip_final_df['GEOMETRY_Northing'].convert_objects(convert_numeric=True).apply(lambda x: x/10)

equip_final_df['GEOMETRY_Easting'] = equip_final_df['GEOMETRY_Easting'].astype('O')
equip_final_df.loc[equip_final_df['GEOMETRY_Easting'].notnull(), 'GEOMETRY_Easting'] = equip_final_df.loc[equip_final_df['GEOMETRY_Easting'].notnull(), 'GEOMETRY_Easting'].astype(np.float64)

equip_final_df['GEOMETRY_Northing'] = equip_final_df['GEOMETRY_Northing'].astype('O')
equip_final_df.loc[equip_final_df['GEOMETRY_Northing'].notnull(), 'GEOMETRY_Northing'] = equip_final_df.loc[equip_final_df['GEOMETRY_Northing'].notnull(), 'GEOMETRY_Northing'].astype(np.float64)

# conv_csv(loc,equip_final_df,'equip_final_df')



C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipyker

In [12]:
# merging INS station details with PIPER equipment details

equip_final_df = equip_final_df.rename(columns = {'BT_PLANT_1141_CODE':'STTN_1141_CODE'})
ins_piper_df = pd.merge(ins_STTN_df,equip_final_df,on = 'STTN_1141_CODE', how = 'outer')
# conv_csv(loc,ins_piper_df,'ins_piper_df')

# removing the decimal values in easting and northing values
ins_piper_df_geo = ins_piper_df[(np.isclose(ins_piper_df['STTN_EASTING'].values[:, None],ins_piper_df['GEOMETRY_Easting'].values[:, None],atol = 1)) & (np.isclose(ins_piper_df['STTN_NORTHING'].values[:, None],ins_piper_df['GEOMETRY_Northing'].values[:, None],atol = 1))]
# conv_csv(loc,ins_piper_df_geo,'ins_piper_df_geo')

In [13]:
# merging piper terminating cables, converting to numeric and finding the count
Equ_Ter_df = pd.merge(equip_final_df,Terminating_df,on = 'EQUIPMENT_NAME', how = 'left')
Equ_Ter_df['BT_CONDUCTOR_NUMBER'] = Equ_Ter_df['BT_CONDUCTOR_NUMBER'].convert_objects(convert_numeric=True)
Equ_Ter_count_df =Equ_Ter_df.groupby(['STTN_1141_CODE']).agg({'BT_CONDUCTOR_NUMBER':np.sum}).reset_index()

# merging piper originating cables, converting to numeric and finding the count
Equ_Org_df = pd.merge(equip_final_df,Originating_df,on = 'EQUIPMENT_NAME', how = 'left')
Equ_Org_df['BT_CONDUCTOR_NUMBER'] = Equ_Org_df['BT_CONDUCTOR_NUMBER'].convert_objects(convert_numeric=True)
Equ_Org_count_df = Equ_Org_df.groupby(['STTN_1141_CODE']).agg({'BT_CONDUCTOR_NUMBER':np.sum}).reset_index()

C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [14]:
# geo mismatch
geo_mis_df = ins_piper_df[~ins_piper_df['STTN_1141_CODE'].isin(ins_piper_df_geo['STTN_1141_CODE'].tolist())]
# conv_csv(o_loc,geo_mis_df,'geo_mis_df')

In [15]:
ins_ter_count_df =ins_ter_df.groupby(['STTN_1141_CODE']).agg({'Total Fibres':np.sum}).reset_index()
ins_org_count_df =ins_org_df.groupby(['STTN_1141_CODE']).agg({'Total Fibres':np.sum}).reset_index()

In [16]:
ins_ter_cond_count_df =pd.merge(ins_ter_df,ins_ter_count_df, on = 'STTN_1141_CODE',how =  'right')

ins_org_cond_count_df =pd.merge(ins_org_df,ins_org_count_df, on = 'STTN_1141_CODE',how =  'right')

ins_ter_cond_count_df = ins_ter_cond_count_df[['STTN_1141_CODE','SubarCJ','Total Fibres_y']]
ins_org_cond_count_df = ins_org_cond_count_df[['STTN_1141_CODE','SubarCJ','Total Fibres_y']]

# conductor count based on cable names - Piper
equip_final_merge_org_df = pd.merge(equip_final_df,Originating_df,on = 'EQUIPMENT_NAME',how = 'left')

equip_final_merge_Ter_df = pd.merge(equip_final_df,Terminating_df,on = 'EQUIPMENT_NAME',how = 'left')

equip_final_merge_org_df['Cable Name'] = equip_final_merge_org_df['BT_CABLE_NAME']+" " +equip_final_merge_org_df['BT_CABLE_NAME_DESIGNATION']

equip_final_merge_Ter_df['Cable Name'] = equip_final_merge_Ter_df['BT_CABLE_NAME']+" " +equip_final_merge_Ter_df['BT_CABLE_NAME_DESIGNATION']

In [17]:
del equip_final_merge_org_df['Cable Name']
del equip_final_merge_Ter_df['Cable Name']

In [18]:
# Preparing final reference data frame

# reset index
ins_ter_cond_count_df = ins_ter_cond_count_df.reset_index()
equip_final_merge_Ter_df = equip_final_merge_Ter_df.reset_index()
ins_org_cond_count_df = ins_org_cond_count_df.reset_index()
equip_final_merge_org_df = equip_final_merge_org_df.reset_index()

#remove index
del equip_final_merge_org_df['index']
del equip_final_merge_Ter_df['index']

#rearrange
equip_final_merge_org_df = equip_final_merge_org_df[['BT_CABLE_DESIGNATION','STTN_1141_CODE','BT_CONDUCTOR_NUMBER']]
equip_final_merge_Ter_df = equip_final_merge_Ter_df[['BT_CABLE_DESIGNATION','STTN_1141_CODE','BT_CONDUCTOR_NUMBER']]

# rename piper columns
equip_final_merge_org_df = equip_final_merge_org_df.rename(columns = {'BT_CABLE_DESIGNATION':'SubarCJ','BT_CONDUCTOR_NUMBER':'Total Fibres'})
equip_final_merge_Ter_df = equip_final_merge_Ter_df.rename(columns = {'BT_CABLE_DESIGNATION':'SubarCJ','BT_CONDUCTOR_NUMBER':'Total Fibres'})

# adding columns
equip_final_merge_Ter_df['INS/PIPER'] = 'PIPER'
equip_final_merge_Ter_df['Terminating/Originating'] = 'Terminating'
ins_ter_cond_count_df['INS/PIPER'] = 'INS'
ins_ter_cond_count_df['Terminating/Originating'] = 'Terminating'
equip_final_merge_org_df['INS/PIPER'] = 'PIPER'
equip_final_merge_org_df['Terminating/Originating'] = 'Originating'
ins_org_cond_count_df['INS/PIPER'] = 'INS'
ins_org_cond_count_df['Terminating/Originating'] = 'Originating'

# merging ins and piper df
final_ref_df = equip_final_merge_Ter_df
final_ref_df = final_ref_df.append(ins_ter_cond_count_df)
final_ref_df = final_ref_df.append(equip_final_merge_org_df)
final_ref_df = final_ref_df.append(ins_org_cond_count_df)

# conv_csv(loc,final_ref_df,'final_ref_df')

In [19]:
%%time
# Sorting final dataframe in given/derived(bt_plant) order
final_ref_df2 = final_ref_df[['STTN_1141_CODE','SubarCJ','Total Fibres','INS/PIPER','Terminating/Originating']]
# Defining Core/Spine and Fiber/Cooper
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('ON', na=False),'Cable Network Category'] = 'Spine'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('ON', na=False),'Network Technology'] = 'Fibre'

final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('CJ', na=False),'Cable Network Category'] = 'Core'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('CJ', na=False),'Network Technology'] = 'Copper'

final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LK', na=False),'Cable Network Category'] = 'Access'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LK', na=False),'Network Technology'] = 'Copper'

final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LD', na=False),'Cable Network Category'] = 'Access'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LD', na=False),'Network Technology'] = 'Copper'

final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LM', na=False),'Cable Network Category'] = 'Spine'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('LM', na=False),'Network Technology'] = 'Copper'

final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('DI', na=False),'Cable Network Category'] = 'Unknown'
final_ref_df2.loc[final_ref_df2['SubarCJ'].str.contains('DI', na=False),'Network Technology'] = 'Copper'

final_ref_df2.loc[~final_ref_df2['SubarCJ'].str.contains('CJ|ON|LK|LD|LM|DI', na=False),'Cable Network Category'] = 'Unknown'
final_ref_df2.loc[~final_ref_df2['SubarCJ'].str.contains('CJ|ON|LK|LD|LM|DI', na=False),'Network Technology'] = 'Unknown'

# conv_csv(o_loc,final_ref_df2,'final_ref_df2')

C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


Wall time: 26.4 s


In [57]:
%%time
# loading & creating final output format - Detailed
no_of_DQI = 6
Tnodes = []
DQI = []
Status = []
Value_in_INS = []
Value_in_PIPER = []
DQI_sub = ['Tnode present in INS and PIPER','Tnode is present only in INS','Tnode is present only in PIPeR','Tnode location between PIPeR and INS','Originating fibre counts from Tnode','Terminating fibres counts from Tnode'] 
for i in range(len(bt_plant)):
    for j in range(no_of_DQI):  
        Tnodes.append(bt_plant[i])
        DQI.append(DQI_sub[j])        

final_df_detailed = pd.DataFrame({'Tnodes':Tnodes, 'DQI':DQI})

final_df_detailed['Status'] = pd.Series()
final_df_detailed['Value in INS'] = pd.Series()
final_df_detailed['Value in PIPER'] = pd.Series()

# 1st DQI
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = False) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = False)),'Status'] = 'Success'
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = True) | np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = True)),'Status'] = 'Failed'

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = False) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = False)),'Value in INS'] = final_df_detailed['Tnodes']
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = False) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = False)),'Value in PIPER'] = final_df_detailed['Tnodes']
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = True) | np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = True)),'Value in INS'] = ''
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[0]) & (np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = True) | np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = True)),'Value in PIPER'] = ''

Wall time: 50.7 s


In [58]:

# 2nd DQI
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[1]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = False),'Status'] = 'Success'
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[1]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = True),'Status'] = 'Failed'

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[1]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = False),'Value in INS'] = final_df_detailed['Tnodes']
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[1]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'INS']['STTN_1141_CODE']),invert = True),'Value in INS'] = ''


# 3rd DQI
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[2]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = False),'Status'] = 'Success'
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[2]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = True),'Status'] = 'Failed'

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[2]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = False),'Value in PIPER'] = final_df_detailed['Tnodes']
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[2]) & np.isin(final_df_detailed['Tnodes'],(final_ref_df2[final_ref_df2['INS/PIPER'] == 'PIPER']['STTN_1141_CODE']),invert = True),'Value in PIPER'] = ''


# 4th DQI
ins_piper_df2 = pd.DataFrame({'STTN_1141_CODE':bt_plant})
ins_piper_df2 = pd.merge(ins_piper_df2,ins_piper_df,on = 'STTN_1141_CODE',how = 'left')

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]) & np.isin(final_df_detailed['Tnodes'],geo_mis_df['STTN_1141_CODE'],invert = False),'Status'] = 'Failed'
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]) & np.isin(final_df_detailed['Tnodes'],geo_mis_df['STTN_1141_CODE'],invert = True),'Status'] = 'Success'

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]),'Value in INS']= "Easting - " + ins_piper_df2['STTN_EASTING'].round(0).map(str).values + " Northing - " +  ins_piper_df2['STTN_NORTHING'].round(0).map(str).values
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]),'Value in PIPER'] = "Easting - " + ins_piper_df2['GEOMETRY_Easting'].round().map(str).values + " Northing - " +  ins_piper_df2['GEOMETRY_Northing'].round().map(str).values

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]) & (final_df_detailed['Value in PIPER'] == 'Easting - nan Northing - nan'),'Value in PIPER'] = 'No Geometry Details'
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[3]) & (final_df_detailed['Value in INS'] == 'Easting - nan Northing - nan'),'Value in INS'] = 'No Geometry Details'

#########dfs for 4th and 5th DQI

ins_org_count_df2 = pd.DataFrame({'STTN_1141_CODE':bt_plant})
ins_org_count_df2 = pd.merge(ins_org_count_df2,ins_org_count_df,on = 'STTN_1141_CODE',how = 'left')

Equ_Org_count_df2 = pd.DataFrame({'STTN_1141_CODE':bt_plant})
Equ_Org_count_df2 = pd.merge(Equ_Org_count_df2,Equ_Org_count_df,on = 'STTN_1141_CODE',how = 'left')

ins_ter_count_df2 = pd.DataFrame({'STTN_1141_CODE':bt_plant})
ins_ter_count_df2 = pd.merge(ins_ter_count_df2,ins_ter_count_df,on = 'STTN_1141_CODE',how = 'left')

Equ_Ter_count_df2 = pd.DataFrame({'STTN_1141_CODE':bt_plant})
Equ_Ter_count_df2 = pd.merge(Equ_Ter_count_df2,Equ_Ter_count_df,on = 'STTN_1141_CODE',how = 'left')

# 5th DQI
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[4]),'Value in INS'] = ins_org_count_df['Total Fibres'].fillna('No Cable Details').values

final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[4]),'Value in PIPER'] = Equ_Org_count_df2['BT_CONDUCTOR_NUMBER'].fillna('No Cable Details').values

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[4]) & np.equal(final_df_detailed['Value in INS'],final_df_detailed['Value in PIPER'])),'Status'] = 'Success'

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[4]) & np.not_equal(final_df_detailed['Value in INS'],final_df_detailed['Value in PIPER'])),'Status'] = 'Failed'

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[4]) & ((final_df_detailed['Value in INS']=='No Cable Details') & (final_df_detailed['Value in PIPER'] == 'No Cable Details'))),'Status'] = 'Failed'

# 6th DQI
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[5]),'Value in INS'] = ins_ter_count_df2['Total Fibres'].fillna('No Cable Details').values
final_df_detailed.loc[(final_df_detailed['DQI'] == DQI_sub[5]),'Value in PIPER'] = Equ_Ter_count_df2['BT_CONDUCTOR_NUMBER'].fillna('No Cable Details').values

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[5]) & np.equal(final_df_detailed['Value in INS'],final_df_detailed['Value in PIPER'])),'Status'] = 'Success'

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[5]) & np.not_equal(final_df_detailed['Value in INS'],final_df_detailed['Value in PIPER'])),'Status'] = 'Failed'

final_df_detailed.loc[((final_df_detailed['DQI'] == DQI_sub[5]) & ((final_df_detailed['Value in INS']=='No Cable Details') & (final_df_detailed['Value in PIPER'] == 'No Cable Details'))),'Status'] = 'Failed'

final_df_detailed = final_df_detailed[['Tnodes','DQI','Status','Value in INS','Value in PIPER']]

In [60]:
%%time
# creating scoring sheet

final_df_scoring = final_df_detailed[['Tnodes','DQI','Status']]

final_df_scoring['Success Point'] = 100
final_df_scoring['Failure Point'] = 0

Weightage = []
Weightage_list = [0.3,0,0,0.3,0.2,0.2]
for i in range(len(bt_plant)):
    for j in range(no_of_DQI):  
        Weightage.append(Weightage_list[j])
final_df_scoring['Weightage'] = pd.Series(Weightage)

final_df_scoring = final_df_scoring.rename(columns = {'Status':'DQI Status'})

final_df_scoring['Points post DQI Run'] = pd.Series()
final_df_scoring['Points post DQI Run'][final_df_scoring['DQI Status']=='Success'] = 100
final_df_scoring['Points post DQI Run'][final_df_scoring['DQI Status']=='Failed'] = 0
final_df_scoring['Points post DQI Run'] = final_df_scoring['Points post DQI Run'].astype(int)

final_df_scoring['DQI Score'] = pd.Series()
final_df_scoring['DQI Score'] = final_df_scoring['Weightage']* final_df_scoring['Points post DQI Run']
final_df_scoring['DQI Score'] = final_df_scoring['DQI Score'].astype(int)

final_df_scoring = final_df_scoring[['Tnodes','DQI','Success Point','Failure Point','Weightage','DQI Status','Points post DQI Run','DQI Score']]


C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\611346962\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Wall time: 2.05 s


In [62]:
%%time
# creating format sheet
final_ref_df2['num'] = 1
final_ref_df2_ = final_ref_df2.groupby(['STTN_1141_CODE','Cable Network Category','Network Technology'],sort=False).agg({'num':np.sum}).reset_index()
final_ref_df2_2 = final_df_scoring.groupby(['Tnodes'],sort=False).agg({'DQI Score':np.sum}).reset_index()

final_ref_df2_ = final_ref_df2_.drop_duplicates('STTN_1141_CODE')
del final_ref_df2_['num']

final_df_format = pd.merge(final_ref_df2_,final_ref_df2_2, left_on = 'STTN_1141_CODE', right_on = 'Tnodes', how= 'outer')

final_df_format = final_df_format[final_df_format['STTN_1141_CODE'].notnull()]
final_df_format = final_df_format[['Tnodes','Cable Network Category','Network Technology','DQI Score']]


Wall time: 1.51 s


In [64]:
# Splitting and saving
import math
no_of_DQI = 6
excel_limit = 600000
split_index = final_df_format.shape[0]*no_of_DQI/excel_limit
spliting = int(math.ceil(split_index))

split_no =  final_df_format.shape[0]/spliting


In [26]:
# cable_df = pd.read_csv(r"C:\Users\611346962\Downloads\Project\Inputs\output\Cable_details.csv")

# cable_df1 = cable_df[cable_df['STTN_1141_CODE'].isin(merge_df1['Tnodes'].tolist())]
# cable_df2 = cable_df[cable_df['STTN_1141_CODE'].isin(merge_df2['Tnodes'].tolist())]

In [79]:
# Writing output files to Excel file
def write_excel(final_df_format,final_df_scoring,final_df_detailed,i):
    writer = pd.ExcelWriter(o_file+"DQI_output_file"+str(i+1)+'.xlsx', engine='xlsxwriter')

    final_df_format.to_excel(writer, sheet_name='Format',index=False,encoding='cp1252')
    final_df_scoring.to_excel(writer, sheet_name='Scoring',index=False,encoding='cp1252')
    final_df_detailed.to_excel(writer, sheet_name='Detailed',index=False,encoding='cp1252')
    
    # declaring the df  and excelsheet lists
    df = [final_df_format,final_df_scoring,final_df_detailed]
    excel_sheets = ['Format','Scoring', 'Detailed']

    workbook = writer.book
    worksheet = writer.sheets['Detailed']
    format_red = workbook.add_format({'font_color': '#9C0006'})
    format_green = workbook.add_format({'font_color': '#006100'})
    worksheet.conditional_format('D1:E'+str(1048576), {'type': 'formula','criteria':'=LEFT($C1, '+str(len(final_df_detailed['Tnodes'])+1)+')="Success"','format': format_green })
    worksheet.conditional_format('D1:E'+str(1048576), {'type': 'formula','criteria':'=LEFT($C1, '+str(len(final_df_detailed['Tnodes'])+1)+')="Failed"','format': format_red })

    def get_col_widths(dataframe):
        return [max([len(str(s)) for s in dataframe[col].values] + [len(col)])*1.2 for col in dataframe.columns]
    def set_col_widths(df,ws):
        for i, width in enumerate(get_col_widths(df)):
            ws.set_column(0, i, width)
    def borders(dataframe,ws):
        formater = workbook.add_format({'border':1})
        ws.set_column(0,len(dataframe.columns.values),len(dataframe),formater)
    for i,value in enumerate (excel_sheets):
        ws = writer.sheets[value]
        borders(df[i],ws)
        set_col_widths(df[i],ws)

    writer.save()

In [78]:
for i in range(spliting):
    format_df = final_df_format[(split_no*i):((split_no)*(i+1))]
    scoring_df = final_df_scoring[final_df_scoring['Tnodes'].isin(format_df['Tnodes'].tolist())]
    detailed_df = final_df_detailed[final_df_detailed['Tnodes'].isin(format_df['Tnodes'].tolist())]
    write_excel(format_df,scoring_df,detailed_df,i)

In [28]:
# cable_df1['STTN_1141_CODE'] = cable_df1['STTN_1141_CODE'].str.decode('cp1252')
# cable_df1['Cable Name'] = cable_df1['Cable Name'].str.decode('cp1252')
# cable_df1['SubarCJ'] = cable_df1['SubarCJ'].str.decode('cp1252')

In [29]:
# bt_list = [bt_plant[i:i + 1000] for i in xrange(0, len(bt_plant), 1000)]

# for j in range(len(bt_list)):
#     final_ref2 = pd.DataFrame()
#     for i in range(len(bt_list[j])):
#         final_ref2 = final_ref2.append(final_ref_df2[final_ref_df2['STTN_1141_CODE']== bt_list[j][i]])
#     final_ref2 = final_ref2[['STTN_1141_CODE','SubarCJ','Cable Name','Total Fibres','INS/PIPER','Terminating/Originating']]
#     conv_csv(c_loc,final_ref2,'final_ref'+" "+str(j+1))